# CrewAI C.V. Agents

## The Job Posting

In [ ]:
job_posting_url = "https://app.welcometothejungle.com/dashboard/jobs/oA1SArxV"

## The Goal

1. Analyze the job posting to extract structured requirements.
2. Profile a candidate by querying a knowledge base for data relevant to the job posting
3. Optimize the candidate's CV based on the job requirements and profile data

## The Data Flow

1. Job posting URL → Job Analyst → Structured job requirements
2. Candidate data + Job requirements → Candidate Profiler → Profile analysis  
3. Profile + Job requirements → CV Strategist → Optimization recommendations
4. Optimized data → Template engine → LaTeX → PDF

## The Implementation

*Import the Agents and functions to create the Tasks:*

In [ ]:
from optimizer.agents.job_analyst import job_analyst as analyst
from optimizer.agents.candidate_profiler import candidate_profiler as profiler
from optimizer.agents.cv_strategist import cv_strategist as strategist

from optimizer.tasks.job_analysis_task import create_job_analysis_task
from optimizer.tasks.candidate_profiling_task import create_candidate_profiling_task
from optimizer.tasks.cv_optimization_task import create_cv_optimization_task

*Chunk knowledge base docs and embed them in a vector store:*

In [ ]:
from embedding_tool import KnowledgeBaseEmbedder
embedder = KnowledgeBaseEmbedder()
result = embedder.create_embeddings()

*Verify vector store status:*

In [ ]:
vectorstore = embedder.get_vectorstore()
if vectorstore:
    count = vectorstore._collection.count()
    print(f"✓ Vector store ready: {count} vectors")
else:
    print("✗ Vector store not found")

*Set up the Tasks:*

In [ ]:
job_analysis = create_job_analysis_task(analyst)
candidate_profile = create_candidate_profiling_task(profiler, job_analysis)
cv_optimization = create_cv_optimization_task(strategist, job_analysis, candidate_profile)

*Set up the Crew*

In [ ]:
from crewai import Crew, LLM
from config import get_config

config = get_config()

inputs = {
    "job_posting_url": job_posting_url,
    "cv_data_path": "data/cv.yaml",
}

llm = LLM(
    model=config.crew_manager_model, temperature=config.crew_manager_temperature,
)

crew = Crew(
    agents=[analyst, profiler, strategist],
    tasks=[job_analysis, candidate_profile, cv_optimization],
    llm=llm,
    inputs=inputs,
    verbose=True,
    debug=True,
)

*Release the hounds!*

In [ ]:
result = crew.kickoff(inputs=inputs)